In [38]:
import pandas as pd
import xgboost as xgb
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score, RandomizedSearchCV
import matplotlib.pyplot as plt





In [39]:
train = pd.read_csv('data/datasets/ais_train.csv', sep='|')
train['time'] = pd.to_datetime(train['time'])
train.info()
missing_values = train.isnull().sum()
print("Number of missing values in each column:\n", missing_values)


train.head()

test = pd.read_csv('data/datasets/ais_test.csv', sep=',')
test['time'] = pd.to_datetime(test['time'])
test.head()

vessels = pd.read_csv('data/datasets/vessels.csv', sep='|')
vessels.head()

ports = pd.read_csv('data/datasets/ports.csv', sep='|')
ports.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1522065 entries, 0 to 1522064
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   time       1522065 non-null  datetime64[ns]
 1   cog        1522065 non-null  float64       
 2   sog        1522065 non-null  float64       
 3   rot        1522065 non-null  int64         
 4   heading    1522065 non-null  int64         
 5   navstat    1522065 non-null  int64         
 6   etaRaw     1522065 non-null  object        
 7   latitude   1522065 non-null  float64       
 8   longitude  1522065 non-null  float64       
 9   vesselId   1522065 non-null  object        
 10  portId     1520450 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(3), object(3)
memory usage: 127.7+ MB
Number of missing values in each column:
 time            0
cog             0
sog             0
rot             0
heading         0
navstat         0
etaRaw          0

,portId,name,portLocation,longitude,latitude,UN_LOCODE,countryName,ISO
0,61d36ed80a1807568ff9a064,Port of Algiers,Algiers,3.067222,36.773611,DZALG,Algeria,DZ
1,61d36ed80a1807568ff9a065,Port of Annaba,Annaba,7.772500,36.900556,DZAAE,Algeria,DZ
2,61d36edf0a1807568ff9a070,Port of Oran,Oran,-0.639722,35.712222,DZORN,Algeria,DZ
3,61d36ee00a1807568ff9a072,Port of Skikda,Skikda,6.905833,36.887500,DZSKI,Algeria,DZ
4,61d36ee10a1807568ff9a074,Port of Pago-Pago,Pago-Pago,-170.690556,-14.274167,ASPPG,American Samoa,AS


In [40]:
port_id_mapping = {port_id: idx for idx, port_id in enumerate(train['portId'].unique())}
train['portId'] = train['portId'].map(port_id_mapping)
ports['portId'] = ports['portId'].map(port_id_mapping)

vessel_id_mapping = {vessel_id: idx for idx, vessel_id in enumerate(train['vesselId'].unique())}
train['vesselId'] = train['vesselId'].map(vessel_id_mapping)
vessels['vesselId'] = vessels['vesselId'].map(vessel_id_mapping)
test['vesselId'] = test['vesselId'].map(vessel_id_mapping)

shipping_line_id_mapping = {shipping_line_id: idx for idx, shipping_line_id in enumerate(vessels['shippingLineId'].unique())}
vessels['shippingLineId'] = vessels['shippingLineId'].map(shipping_line_id_mapping)

In [41]:
vessels.head()

,shippingLineId,vesselId,CEU,DWT,GT,NT,vesselType,breadth,depth,draft,enginePower,freshWater,fuel,homePort,length,maxHeight,maxSpeed,maxWidth,rampCapacity,yearBuilt
0,0,599.0,6500,21200.0,58684,17606.0,83.0,32.0,22.20,NaN,0.0,NaN,NaN,OSLO,199.00,5.0,18.6,15.2,150.0,2000
1,1,65.0,4902,12325.0,46800,NaN,83.0,31.0,NaN,NaN,14220.0,NaN,NaN,MONROVIA,182.00,NaN,NaN,NaN,NaN,2006
2,2,640.0,5000,13059.0,46800,NaN,83.0,31.0,NaN,NaN,14220.0,NaN,NaN,SAINT JOHN'S,182.00,NaN,NaN,NaN,NaN,2010
3,3,255.0,4200,12588.0,39362,NaN,83.0,28.0,NaN,NaN,11060.0,NaN,NaN,NaN,167.00,NaN,NaN,NaN,NaN,2011
4,4,68.0,7450,21052.0,75528,24391.0,83.0,37.2,22.23,NaN,13140.0,491.47,3236.78,Panama,199.98,NaN,NaN,NaN,NaN,2018


In [42]:


train = train.merge(ports[['portId', 'latitude', 'longitude']], how='left', left_on='portId', right_on='portId', suffixes=('', '_port'))
train.head()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId,latitude_port,longitude_port
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,01-09 23:00,-34.74370,-57.85130,0,0,-33.5875,-71.618889
1,2024-01-01 00:00:36,109.6,0.0,-6,347,1,12-29 20:00,8.89440,-79.47939,1,1,8.9670,-79.533000
2,2024-01-01 00:01:45,111.0,11.0,0,112,0,01-02 09:00,39.19065,-76.47567,2,2,39.2325,-76.558889
3,2024-01-01 00:03:11,96.4,0.0,0,142,1,12-31 20:00,-34.41189,151.02067,3,3,-34.4625,150.899444
4,2024-01-01 00:03:51,214.0,19.7,0,215,0,01-25 12:00,35.88379,-5.91636,4,4,35.7830,-5.817000


In [43]:
train = train.merge(vessels[['vesselId', 'length', 'shippingLineId', 'maxSpeed', 'breadth', 'DWT']], on='vesselId', how='left')
train['vessel_deep_sea'] = np.where(train['length'] > 200, 1, 0)

num_maxSpeed_nan = train['breadth'].isna().sum()
print(f"Number of NaN values in maxSpeed: {num_maxSpeed_nan}")
train.head()

Number of NaN values in maxSpeed: 14949


,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId,latitude_port,longitude_port,length,shippingLineId,maxSpeed,breadth,DWT,vessel_deep_sea
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,01-09 23:00,-34.74370,-57.85130,0,0,-33.5875,-71.618889,199.00,9,NaN,32.00,21214.0,0
1,2024-01-01 00:00:36,109.6,0.0,-6,347,1,12-29 20:00,8.89440,-79.47939,1,1,8.9670,-79.533000,199.97,6,22.5,32.26,18878.0,0
2,2024-01-01 00:01:45,111.0,11.0,0,112,0,01-02 09:00,39.19065,-76.47567,2,2,39.2325,-76.558889,199.00,14,NaN,32.00,18383.0,0
3,2024-01-01 00:03:11,96.4,0.0,0,142,1,12-31 20:00,-34.41189,151.02067,3,3,-34.4625,150.899444,199.00,5,NaN,32.00,15199.0,0
4,2024-01-01 00:03:51,214.0,19.7,0,215,0,01-25 12:00,35.88379,-5.91636,4,4,35.7830,-5.817000,199.95,6,22.4,32.20,18833.0,0


In [ ]:
from scipy.spatial import ConvexHull

def calculate_area_covered(window):
    """
    Calculate the area covered by a set of latitude and longitude points in the window.
    Uses the convex hull of the points.
    """
    if len(window) < 3:  # Convex hull requires at least 3 points
        return 0

    points = window[['latitude', 'longitude']].dropna().to_numpy()
    hull = ConvexHull(points)
    return hull.volume  # 'volume' gives the area in 2D

In [44]:

def create_five_day_windows(df):
    # Ensure that 'time' column is in datetime format
    df['time'] = pd.to_datetime(df['time'])
    
    # Sort data to ensure time sequence within each vessel
    df = df.sort_values(by=['vesselId', 'time']).reset_index(drop=True)
    
    window_size_days = 5
    windows = []

    for vessel_id, group in df.groupby('vesselId'):
        group = group.sort_values(by='time')
        unique_dates = group['time'].dt.date.unique()

        for start_idx in range(len(unique_dates) - window_size_days + 1):
            start_date = unique_dates[start_idx]
            end_date = unique_dates[start_idx + window_size_days - 1]
            window = group[(group['time'].dt.date >= start_date) & (group['time'].dt.date <= end_date)]

            # Skip empty windows
            if len(window) == 0:
                continue

            area_covered = calculate_area_covered(window)

            # Reference row: the first row in the window
            reference_row = window.iloc[0]

            # Append each row in this window with reference features from the first row
            for _, row in window.iterrows():
                windows.append({
                    'vesselId': row['vesselId'],
                    'time': row['time'],
                    'latitude': row['latitude'],
                    'longitude': row['longitude'],
                    'cog': row['cog'],
                    'sog': row['sog'],
                    'rot': row['rot'],
                    'under_way': row['under_way'],
                    # Features based on the first row in the window
                    'latitude_first': reference_row['latitude'],
                    'longitude_first': reference_row['longitude'],
                    'cog_first': reference_row['cog'],
                    'sog_first': reference_row['sog'],
                    'rot_first': reference_row['rot'],
                    'under_way_first': reference_row['under_way'],
                    'time_since_start': (row['time'] - reference_row['time']).total_seconds(),
                    'area_covered': area_covered
                })

    return pd.DataFrame(windows)

In [45]:
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a)) 
    r = 6371  # Radius of Earth in kilometers
    return r * c  # Distance in kilometers

### Feature engineering 

In [46]:
def feature_engineering(train):
    train = train.copy()
    train = train.sort_values(by=['vesselId', 'time']).reset_index(drop=True)

    # # Handle NaNs
    # train[['latitude', 'longitude', 'cog', 'time']] = (
    #     train.groupby('vesselId')[['latitude', 'longitude', 'cog', 'time']].apply(lambda x: x.ffill().bfill())
    # ).reset_index(drop=True)

    features = pd.DataFrame()
    features['vesselId'] = train['vesselId']
    features['time'] = train['time'] 
    features['latitude'] = train['latitude']
    features['longitude'] = train['longitude']
    features['cog'] = train['cog']
    features['sog'] = train['sog']
    features['rot'] = train['rot']
    features['under_way'] = train['navstat'].isin([0, 8]).astype(int)
    features['cog_rad'] = np.radians(features['cog'])
    features['cog_sin'] = np.sin(features['cog_rad'])
    features['cog_cos'] = np.cos(features['cog_rad'])

    # Additional angle transformations
   

    # Calculate area covered within the time window
    features = create_five_day_windows(features)

 
    # Additional time-based features
    features['month'] = features['time'].dt.month
    features['day'] = features['time'].dt.day
    features['hour'] = features['time'].dt.hour
    features['minute'] = features['time'].dt.minute
    features['day_of_week'] = features['time'].dt.dayofweek

    return features

train_features = feature_engineering(train)

KeyboardInterrupt: 

In [30]:
nan_values = train_features.isnull().sum()
print(nan_values)

vesselId            0
time                0
latitude            0
longitude           0
cog                 0
sog                 0
rot                 0
under_way           0
latitude_first      0
longitude_first     0
cog_first           0
sog_first           0
rot_first           0
under_way_first     0
time_since_start    0
month               0
day                 0
hour                0
minute              0
day_of_week         0
dtype: int64


In [31]:
train_features.head()

,vesselId,time,latitude,longitude,cog,sog,rot,under_way,latitude_first,longitude_first,cog_first,sog_first,rot_first,under_way_first,time_since_start,month,day,hour,minute,day_of_week
0,0,2024-01-01 00:00:25,-34.74370,-57.85130,284.0,0.7,0,1,-34.7437,-57.8513,284.0,0.7,0,1,0.0,1,1,0,0,0
1,0,2024-01-01 06:09:08,-35.16787,-56.77210,92.8,14.2,0,1,-34.7437,-57.8513,284.0,0.7,0,1,22123.0,1,1,6,9,0
2,0,2024-01-01 06:38:19,-35.16863,-56.63185,90.5,14.3,0,1,-34.7437,-57.8513,284.0,0.7,0,1,23874.0,1,1,6,38,0
3,0,2024-01-01 06:58:55,-35.16805,-56.53190,88.2,14.3,0,1,-34.7437,-57.8513,284.0,0.7,0,1,25110.0,1,1,6,58,0
4,0,2024-01-01 07:15:56,-35.16715,-56.45306,88.3,12.3,0,1,-34.7437,-57.8513,284.0,0.7,0,1,26131.0,1,1,7,15,0


In [32]:
def find_last_features(features):
    # Assuming that the last row for every vesselId is the most recent
    last_features = features.groupby('vesselId').last().reset_index()
    return last_features

last_features = find_last_features(train_features)
last_features.head()


,vesselId,time,latitude,longitude,cog,sog,rot,under_way,latitude_first,longitude_first,cog_first,sog_first,rot_first,under_way_first,time_since_start,month,day,hour,minute,day_of_week
0,0,2024-05-01 12:41:57,34.57936,128.99926,221.5,15.5,0,1,47.54253,-122.52499,53.8,0.4,0,0,1340708.0,5,1,12,41,2
1,1,2024-04-30 10:12:01,1.24460,103.39997,305.1,15.7,11,1,24.98448,55.06391,81.6,0.1,0,0,2195733.0,4,30,10,12,1
2,2,2024-05-07 23:59:02,18.13873,-69.74863,176.0,0.4,0,1,40.69757,-74.15090,0.0,0.0,0,0,603974.0,5,7,23,59,1
3,3,2024-04-26 13:29:25,41.64055,143.29942,87.6,14.4,0,1,36.84741,125.83660,60.8,9.7,0,1,455585.0,4,26,13,29,4
4,4,2024-05-07 12:28:39,26.58710,121.27831,39.1,12.7,0,1,1.26628,103.85164,341.0,0.0,0,0,562766.0,5,7,12,28,1


### Train the model

In [33]:
# # Define features and target
y = last_features[['latitude', 'longitude']]  
X = last_features.drop(columns=['latitude', 'longitude', 'time', 'cog']) 

X.head()

,vesselId,sog,rot,under_way,latitude_first,longitude_first,cog_first,sog_first,rot_first,under_way_first,time_since_start,month,day,hour,minute,day_of_week
0,0,15.5,0,1,47.54253,-122.52499,53.8,0.4,0,0,1340708.0,5,1,12,41,2
1,1,15.7,11,1,24.98448,55.06391,81.6,0.1,0,0,2195733.0,4,30,10,12,1
2,2,0.4,0,1,40.69757,-74.15090,0.0,0.0,0,0,603974.0,5,7,23,59,1
3,3,14.4,0,1,36.84741,125.83660,60.8,9.7,0,1,455585.0,4,26,13,29,4
4,4,12.7,0,1,1.26628,103.85164,341.0,0.0,0,0,562766.0,5,7,12,28,1


In [34]:
# model = RandomForestRegressor(random_state=42)

# # Define the parameter grid for Random Forest
# param_grid = {
#     'n_estimators': [100, 300],            # Number of trees in the forest
#     'max_depth': [5, 7, 10, None],             # Maximum depth of the tree
#     'min_samples_split': [2, 5],           # Minimum number of samples required to split an internal node
#     'min_samples_leaf': [1, 2],            # Minimum number of samples required to be at a leaf node
#     'max_features': ['auto', 'sqrt'],      # The number of features to consider when looking for the best split
#     'bootstrap': [True, False]             # Whether bootstrap samples are used when building trees
# }

# # Perform Randomized Search
# random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
#                                    n_iter=20, scoring='neg_mean_squared_error', cv=5, verbose=1, random_state=42)
# random_search.fit(X, y)

# best_model = random_search.best_estimator_

# # Output the best parameters and score
# print(f"Best parameters: {random_search.best_params_}")
# print(f"Best score (negative mean squared error): {-random_search.best_score_}")

best_model = xgb.XGBRegressor()

# Fit the model
best_model.fit(X, y)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [35]:
X.head()

,vesselId,sog,rot,under_way,latitude_first,longitude_first,cog_first,sog_first,rot_first,under_way_first,time_since_start,month,day,hour,minute,day_of_week
0,0,15.5,0,1,47.54253,-122.52499,53.8,0.4,0,0,1340708.0,5,1,12,41,2
1,1,15.7,11,1,24.98448,55.06391,81.6,0.1,0,0,2195733.0,4,30,10,12,1
2,2,0.4,0,1,40.69757,-74.15090,0.0,0.0,0,0,603974.0,5,7,23,59,1
3,3,14.4,0,1,36.84741,125.83660,60.8,9.7,0,1,455585.0,4,26,13,29,4
4,4,12.7,0,1,1.26628,103.85164,341.0,0.0,0,0,562766.0,5,7,12,28,1


In [36]:
print(X.keys())
'''
['vesselId', 'cog', 'sog', 'under_way', 'latitude_first',
       'longitude_first', 'cog_first', 'sog_first', 'under_way_first',
       'time_since_start', 'month', 'day', 'hour', 'minute']
'''

### Prepare test data for predictions
def prepare_test_for_predictions(test, last_features):
    test = test.copy()
    prepared_test = pd.DataFrame()

    # Create a time column in last features
    last_features['year'] = 2024
    last_features['time'] = pd.to_datetime(
        last_features[['year', 'month', 'day', 'hour', 'minute']]
    )

    # Add the columns vesselId and time
    prepared_test['vesselId'] = test['vesselId']
    prepared_test['time'] = test['time']

    # For each vessel, add the last seen features to the prepared test
    prepared_test = prepared_test.merge(last_features, on='vesselId', how='left', suffixes=('', '_last'))

    print(prepared_test.head())

    # Move the last_features to the refrence row in the windows
    prepared_test['latitude_first'] = prepared_test['latitude']
    prepared_test['longitude_first'] = prepared_test['longitude']
    prepared_test['cog_sin_first'] = prepared_test['cog_sin']
    prepared_test['cog_cos_first'] = prepared_test['cog_sin']
    prepared_test['sog_first'] = prepared_test['sog']
    prepared_test['under_way_first'] = prepared_test['under_way']
    prepared_test['time_since_start'] = (prepared_test['time'] - prepared_test['time_last']).dt.total_seconds()

    # Split the time column into month, day, hour, minute and second columns
    prepared_test['month'] = test['time'].dt.month
    prepared_test['day'] = test['time'].dt.day
    prepared_test['hour'] = test['time'].dt.hour
    prepared_test['minute'] = test['time'].dt.minute
    # prepared_test['second'] = test['time'].dt.second

    prepared_test.drop('time', axis=1, inplace=True)
    prepared_test.drop('time_last', axis=1, inplace=True)
    prepared_test.drop('year', axis=1, inplace=True)

    # Reorder the columns
    # prepared_test = prepared_test[['vesselId', 'cog_sin', 'cog_cos', 'sog', 'under_way',
    #                                'latitude_first', 'longitude_first', 'cog_sin_first', 'cog_cos_first', 'sog_first',
    #                                  'under_way_first', 'time_since_start', 'month', 'day',
    #                                'hour', 'minute']]

    return prepared_test

test_df = prepare_test_for_predictions(test, last_features)
print(test_df.head())
print(test_df.shape)

Index(['vesselId', 'sog', 'rot', 'under_way', 'latitude_first',
       'longitude_first', 'cog_first', 'sog_first', 'rot_first',
       'under_way_first', 'time_since_start', 'month', 'day', 'hour', 'minute',
       'day_of_week'],
      dtype='object')
   vesselId                time           time_last  latitude  longitude  \
0       412 2024-05-08 00:03:16 2024-05-07 23:48:00  31.14647  -81.49789   
1       373 2024-05-08 00:06:17 2024-05-07 23:57:00  14.81694  120.29625   
2       181 2024-05-08 00:10:02 2024-05-07 23:59:00  38.27895   10.78280   
3         8 2024-05-08 00:10:34 2024-05-07 23:52:00 -43.53785  172.83522   
4        65 2024-05-08 00:12:27 2024-05-07 23:51:00  48.53320   -6.12003   

     cog   sog  rot  under_way  latitude_first  ...  sog_first  rot_first  \
0  179.6   0.0    0          0        30.93466  ...        0.0          2   
1   24.7   0.0    0          0        35.44624  ...        0.0          0   
2    8.0  18.7    0          1        38.00859  ...       

In [37]:
predictions = best_model.predict(test_df)
print(predictions)
print(predictions.shape)

ValueError: feature_names mismatch: ['vesselId', 'sog', 'rot', 'under_way', 'latitude_first', 'longitude_first', 'cog_first', 'sog_first', 'rot_first', 'under_way_first', 'time_since_start', 'month', 'day', 'hour', 'minute', 'day_of_week'] ['vesselId', 'cog', 'sog', 'under_way', 'latitude_first', 'longitude_first', 'cog_first', 'sog_first', 'under_way_first', 'time_since_start', 'month', 'day', 'hour', 'minute']
expected rot_first, day_of_week, rot in input data
training data did not have the following fields: cog

In [ ]:
predictions_df = pd.DataFrame(predictions, columns=['latitude_predicted', 'longitude_predicted'])
predictions_df['ID'] = range(len(predictions_df))
predictions_df = predictions_df[['ID', 'longitude_predicted', 'latitude_predicted']]

# Save to CSV
predictions_df.to_csv('predictions_ing_area.csv', index=False)

In [1]:
merged_df = pd.merge(test, predictions_df, on='ID', how='left')
merged_df.drop(['ID', 'scaling_factor'])
merged_df.head()

NameError: name 'pd' is not defined

In [23]:
predictions_df.to_csv('data/submissions/predictions_3_area.csv', index=False)

In [ ]:
merged_df.to_csv('data/submissions/boat_plotting.csv')